# 全データセット活用実験 - 結果分析

この実験では、50,000サンプルから574,945サンプル（全データ）にスケールアップし、データフォーマットをCSVからParquetに最適化しました。

In [ ]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# 日本語フォント設定
plt.rcParams["font.family"] = "DejaVu Sans"
sns.set_style("whitegrid")
plt.figure(figsize=(12, 8))

## 1. 実験結果の読み込み

In [ ]:
# 実験結果を読み込み
with open("../results/comprehensive_benchmark.json") as f:
    results = json.load(f)

print("📊 実験概要")
print(f"実験名: {results['experiment_name']}")
print(f"実施日時: {results['timestamp']}")
print()
print("🎯 主要結果:")
summary = results["summary"]
print(f"ベースライン精度: {summary['baseline_accuracy']:.4f}")
print(f"最適化後精度: {summary['optimized_accuracy']:.4f}")
print(
    f"精度向上: +{summary['accuracy_improvement']:.4f} ({summary['improvement_percentage']:.1f}%)"
)
print(f"データスケール: {summary['data_scale_increase']:.1f}倍")
print(f"特徴量スケール: {summary['feature_scale_increase']:.1f}倍")

## 2. データ最適化効果の分析

In [ ]:
# データ最適化の効果
data_opt = results["data_optimization"]

print("💾 データ最適化効果:")
print(
    f"ファイルサイズ: {data_opt['csv_size_mb']:.1f}MB → {data_opt['parquet_size_mb']:.1f}MB"
)
print(f"圧縮率: {data_opt['compression_ratio']:.1f}倍")
print(
    f"読み込み時間: {data_opt['csv_load_time']:.2f}秒 → {data_opt['parquet_load_time']:.2f}秒"
)
print(f"速度向上: {data_opt['speedup_factor']:.1f}倍")

# 可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# ファイルサイズ比較
formats = ["CSV", "Parquet"]
sizes = [data_opt["csv_size_mb"], data_opt["parquet_size_mb"]]
colors = ["lightcoral", "lightgreen"]

bars1 = ax1.bar(formats, sizes, color=colors)
ax1.set_ylabel("File Size (MB)")
ax1.set_title("File Size Comparison")
for bar, size in zip(bars1, sizes, strict=False):
    ax1.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 20,
        f"{size:.0f}MB",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

# 読み込み時間比較
times = [data_opt["csv_load_time"], data_opt["parquet_load_time"]]
bars2 = ax2.bar(formats, times, color=colors)
ax2.set_ylabel("Loading Time (seconds)")
ax2.set_title("Loading Speed Comparison")
for bar, time_val in zip(bars2, times, strict=False):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.1,
        f"{time_val:.2f}s",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

## 3. 精度向上の分析

In [ ]:
# 精度向上の可視化
experiments = ["Quick Baseline\n(50k samples)", "Full Dataset\n(575k samples)"]
accuracies = [summary["baseline_accuracy"], summary["optimized_accuracy"]]
samples = [50000, 574945]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 精度比較
colors = ["lightblue", "lightgreen"]
bars1 = ax1.bar(experiments, accuracies, color=colors)
ax1.set_ylabel("Validation Accuracy")
ax1.set_title("Accuracy Improvement")
ax1.set_ylim(0.7, 0.85)

for bar, acc in zip(bars1, accuracies, strict=False):
    ax1.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.002,
        f"{acc:.3f}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

# 改善率を矢印で表示
improvement = summary["accuracy_improvement"]
ax1.annotate(
    f"+{improvement:.4f}\n(+{summary['improvement_percentage']:.1f}%)",
    xy=(0.5, (accuracies[0] + accuracies[1]) / 2),
    xytext=(0.5, 0.82),
    ha="center",
    va="center",
    fontsize=12,
    fontweight="bold",
    color="red",
    arrowprops=dict(arrowstyle="<->", color="red", lw=2),
)

# サンプル数比較
bars2 = ax2.bar(experiments, samples, color=colors)
ax2.set_ylabel("Number of Samples")
ax2.set_title("Dataset Scale Comparison")
ax2.ticklabel_format(style="scientific", axis="y", scilimits=(0, 0))

for bar, sample in zip(bars2, samples, strict=False):
    ax2.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + max(samples) * 0.02,
        f"{sample:,}",
        ha="center",
        va="bottom",
        fontweight="bold",
    )

plt.tight_layout()
plt.show()

## 4. スケーリング効果の分析

In [ ]:
# スケーリング効果の詳細分析
print("📈 スケーリング効果分析")
print("=" * 30)

# データ効率性の計算
baseline_samples = 50000
full_samples = 574945
scale_factor = full_samples / baseline_samples

baseline_acc = summary["baseline_accuracy"]
full_acc = summary["optimized_accuracy"]
acc_improvement = summary["improvement_percentage"]

# データ効率性指標
data_efficiency = acc_improvement / (scale_factor - 1)  # 追加データ1倍あたりの精度向上

print(f"データスケール: {scale_factor:.1f}倍 ({baseline_samples:,} → {full_samples:,})")
print(f"精度向上: {acc_improvement:.1f}%")
print(f"データ効率性: {data_efficiency:.2f}% / 1倍追加データ")

# スケーリング曲線の推定
sample_ratios = np.array([1, 2, 5, 10, 11.5])  # 実際のスケール比（11.5倍）
estimated_accuracies = np.array(
    [
        baseline_acc,
        baseline_acc + 0.02,  # 推定値
        baseline_acc + 0.05,  # 推定値
        baseline_acc + 0.08,  # 推定値
        full_acc,  # 実際の値
    ]
)

plt.figure(figsize=(10, 6))
plt.plot(
    sample_ratios[:-1],
    estimated_accuracies[:-1],
    "b--o",
    label="Estimated scaling curve",
    alpha=0.7,
)
plt.plot(
    sample_ratios[-1],
    estimated_accuracies[-1],
    "ro",
    markersize=10,
    label="Actual result",
)
plt.axhline(y=baseline_acc, color="gray", linestyle=":", alpha=0.7, label="Baseline")

plt.xlabel("Data Scale Factor")
plt.ylabel("Validation Accuracy")
plt.title("Data Scaling Effect on Model Performance")
plt.legend()
plt.grid(True, alpha=0.3)

# 実際の結果を強調
plt.annotate(
    f"Actual: {full_acc:.3f}\n(+{acc_improvement:.1f}%)",
    xy=(sample_ratios[-1], estimated_accuracies[-1]),
    xytext=(sample_ratios[-1] + 1, estimated_accuracies[-1] + 0.01),
    fontsize=12,
    fontweight="bold",
    arrowprops=dict(arrowstyle="->", color="red"),
)

plt.tight_layout()
plt.show()

## 5. 提出ファイルの分析

In [ ]:
# 提出ファイルの比較
print("📄 提出ファイル分析")
print("=" * 25)

# ベースライン提出
baseline_submission = pd.read_csv("../submissions/quick_baseline_20250629_133054.csv")
print(f"ベースライン提出: {baseline_submission.shape}")
print("予測クラス分布:")
print(baseline_submission["behavior"].value_counts())

print("\n" + "-" * 25)

# 全データセット提出
full_submission = pd.read_csv(
    "../submissions/full_dataset_quick_randomforest_20250629_152111.csv"
)
print(f"全データセット提出: {full_submission.shape}")
print("予測クラス分布:")
print(full_submission["behavior"].value_counts())

# 予測分布の比較
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# ベースライン
baseline_counts = baseline_submission["behavior"].value_counts()
ax1.pie(
    baseline_counts.values,
    labels=baseline_counts.index,
    autopct="%1.1f%%",
    startangle=90,
)
ax1.set_title(f"Baseline Predictions\n(Accuracy: {baseline_acc:.3f})")

# 全データセット
full_counts = full_submission["behavior"].value_counts()
ax2.pie(full_counts.values, labels=full_counts.index, autopct="%1.1f%%", startangle=90)
ax2.set_title(f"Full Dataset Predictions\n(Accuracy: {full_acc:.3f})")

plt.tight_layout()
plt.show()

## 6. 実験総括と次のステップ

In [ ]:
print("🎉 実験総括")
print("=" * 40)
print("\n✅ 達成された成果:")
print(
    f"  1. 精度大幅向上: {baseline_acc:.3f} → {full_acc:.3f} (+{acc_improvement:.1f}%)"
)
print(f"  2. データ処理高速化: 読み込み速度 {data_opt['speedup_factor']:.1f}倍向上")
print(
    f"  3. ストレージ効率化: ファイルサイズ {data_opt['compression_ratio']:.1f}倍圧縮"
)
print(f"  4. スケーラビリティ実証: {scale_factor:.1f}倍データで安定動作")

print("\n🎯 目標達成状況:")
target_accuracy = 0.78  # Issue #2の目標
achieved = "✅" if full_acc >= target_accuracy else "⚠️"
print(f"  目標精度 ({target_accuracy:.2f}): {achieved} 達成 ({full_acc:.3f})")

print("\n🚀 次のステップ (Phase 1 継続):")
print("  1. 特徴量エンジニアリング実験")
print("     - 時系列特徴量 (移動平均、差分、トレンド)")
print("     - 統計的特徴量 (分散、歪度、尖度)")
print("     - ドメイン特徴量 (センサー間相関、振幅)")
print("  2. データ前処理最適化実験")
print("     - 外れ値除去")
print("     - 正規化手法の比較")
print("     - クラス不均衡対策の改善")

print("\n📊 期待される効果:")
expected_improvement = 0.05  # 追加5%の改善を期待
expected_accuracy = full_acc + expected_improvement
print(f"  Phase 1完了目標: {expected_accuracy:.3f} (現在+{expected_improvement:.3f})")
print("  Phase 2開始条件: 80%以上の精度達成")

print("\n✨ この実験により、Phase 1の基盤が確立されました！")